In [505]:
from bs4 import BeautifulSoup
import re
import requests
import json
f = open("API.txt", "r")
key = f.readline()
f.close()

In [506]:
def get_top_200_population_city():
    url_city = "https://www.michigan-demographics.com/cities_by_population"
    page = requests.get(url_city)
    soup = BeautifulSoup(page.content, 'html.parser')
    city_table = soup.find_all("table", class_="ranklist")
    city_list = [] 
    for i in city_table: 
        name = i.find_all('td')
        for j in name:
            if re.search(r'[~^0-9]',j.text) != None:
                pass
            else:
                text = re.sub('\n','',j.text)
                text = re.sub('          ','',text)
                text = re.sub('      ','',text)
                text = re.sub(' city','',text)
                text = re.sub(' City','',text)
                city_list.append(text)
    top200_city_population = city_list[0:200]
    return top200_city_population  

In [696]:
def get_restaurant(location,offset):
    url='https://api.yelp.com/v3/businesses/search'
    headers = {
    "accept": "application/json",
    "Authorization": key
    }
    params = {'term':'restaurants','location':str(location) + "Michigan","radius" : 35000,"limit":50\
              ,"offset":offset,"locale":"en_US"}

    response = requests.get(url, params=params, headers=headers)
    result = json.loads(response.text)["businesses"]
    return (result)

In [508]:
class location: 
    def __init__(self, name="None",price_tree = []):
        self.name = name
        self.price_tree = price_tree
    def __str__ (self):
        return "location : " + self.name
        

In [509]:
class price: 
    def __init__(self,name="None",rest_tree = []):
        self.name = name
        self.rest_tree = rest_tree
    def __str__ (self):
        return "price : " + self.name

In [510]:
class restaurant:
    def __init__(self, name="None",price= "None",review_count="None",rating = "None",categories = "None" ,id_res= 'None', city="None", location="None",\
                 cord = 'None',url = "None",json = None):
        if json != None:
            self.name = json['name']
            try:
                self.price = json['price']
            except:
                self.price = "None"
            self.review_count = json['review_count']
            self.rating = json['rating']
            self.categories = []
            for tag in json['categories']:
                self.categories.append(tag["title"])
            self.id_res = ['id_res']
            self.city = json['location']['city']
            self.location = json['location']["display_address"]
            self.cord = json['coordinates']
            self.url = json['url']
        else:
            self.name = name
            self.price = price
            self.review_count = review_count
            self.rating = rating
            self.categories = categories
            self.id_res = id_res
            self.city = city
            self.location = location
            self.cord = cord
            self.url = url
    
    def get_categories(self,categories):
        if categories == "None":
            return None
        else:
            str_cate = ""
            for i in categories:
                str_cate = str_cate + i + "; "
        return str_cate
    
    def __str__ (self):
        return "name : " + self.name +"\n"+ "price : " + self.price +"\n"+  "review_count : " + str(self.review_count) +"\n"+ "rating : " + str(self.rating) +"\n"+ "categories : " + self.get_categories(self.categories) +"\n"+ "url : " + self.url +"\n"
                
        

In [639]:
def cache_data(city_list):
    dump_dict ={}
    for city in city_list:
            temp_list = []
            for offset in range(0,1000,50):
                rest_list_json = get_restaurant(city,offset)
                if rest_list_json == []:
                    break
                else:
                    temp_list = temp_list + rest_list_json
            dump_dict[str(city)] = temp_list
            
    with open("restaurant.json", "w") as outfile:
        json.dump(dump_dict, outfile)
            

In [644]:
def tree_build_with_cache(city_list,cache_file):
    f = open(cache_file)
    cache_file = json.load(f)
    f.close()
    
    tree_root = []
    for city in city_list:
        
        price_1_node = price(name = "$",rest_tree=[])
        price_2_node = price(name = "$$",rest_tree=[])
        price_3_node = price(name = "$$$",rest_tree=[])
        price_4_node = price(name = "$$$$",rest_tree=[])
        price_5_node = price(name = "NA",rest_tree=[])
        
        cache_file
        rest_list_json = cache_file[str(city)]
        
        for rest_item_json in rest_list_json:
            rest_node = restaurant(json = rest_item_json)
            if  rest_node.price ==  "$":
                price_1_node.rest_tree.append(rest_node)
            elif rest_node.price == "$$":
                price_2_node.rest_tree.append(rest_node)
            elif rest_node.price == "$$$":
                price_3_node.rest_tree.append(rest_node)
            elif rest_node.price == "$$$$":
                price_4_node.rest_tree.append(rest_node)
            elif rest_node.price == "None":
                price_5_node.rest_tree.append(rest_node)
        
        city_node = location(name = city)
        city_node.price_tree = [price_1_node,price_2_node,price_3_node,price_4_node,price_5_node]
        tree_root.append(city_node)
    return tree_root